# Analog Input with AITask

This notebook demonstrates analog input data acquisition using `nidaqwrapper.AITask`.

**Covers:**
- Voltage channel acquisition
- Accelerometer (IEPE) channel configuration
- Multi-channel synchronous reads
- Context manager usage
- Data shape conventions

**Hardware independence:** The first cell detects whether NI-DAQmx hardware is available. If not, it sets up a mock environment so all examples run without physical hardware.

In [ ]:
# Environment setup: detect hardware or enable mock mode
import sys
from unittest.mock import MagicMock, patch
import numpy as np

try:
    import nidaqmx
    HW_AVAILABLE = True
    print("Running with real NI-DAQmx hardware.")
except ImportError:
    HW_AVAILABLE = False
    print("NI-DAQmx not available — enabling mock mode.")
    
    # Create mock nidaqmx module
    mock_nidaqmx = MagicMock()
    
    # Mock constants
    mock_constants = MagicMock()
    mock_constants.AcquisitionType.CONTINUOUS = MagicMock()
    mock_constants.TerminalConfiguration.DEFAULT = MagicMock()
    mock_constants.VoltageUnits.VOLTS = MagicMock()
    mock_constants.VoltageUnits.FROM_CUSTOM_SCALE = MagicMock()
    mock_constants.AccelUnits.G = MagicMock()
    mock_constants.AccelUnits.METERS_PER_SECOND_SQUARED = MagicMock()
    mock_constants.AccelSensitivityUnits.MILLIVOLTS_PER_G = MagicMock()
    mock_nidaqmx.constants = mock_constants
    
    # Mock Device
    mock_device = MagicMock()
    mock_device.name = "Dev1"
    mock_device.product_type = "PCIe-6320 (Simulated)"
    
    # Mock System
    mock_system = MagicMock()
    mock_system.devices = [mock_device]
    mock_system.tasks.task_names = []
    mock_nidaqmx.system.System.local.return_value = mock_system
    
    # Mock Task
    def create_mock_task(new_task_name=None):
        task = MagicMock()
        task.name = new_task_name or "MockTask"
        task.channel_names = []
        task.ai_channels = []
        task.timing.samp_clk_rate = 25600.0
        task.timing.samp_quant_samp_mode = mock_constants.AcquisitionType.CONTINUOUS
        
        # Mock add_ai_voltage_chan
        def add_voltage_chan(**kwargs):
            ch_name = kwargs.get('name_to_assign_to_channel', 'ai0')
            task.channel_names.append(ch_name)
            mock_channel = MagicMock()
            mock_channel.physical_channel.name = kwargs.get('physical_channel', 'Dev1/ai0')
            task.ai_channels.append(mock_channel)
        
        task.ai_channels.add_ai_voltage_chan = add_voltage_chan
        task.ai_channels.add_ai_accel_chan = add_voltage_chan
        task.ai_channels.add_ai_force_iepe_chan = add_voltage_chan
        
        # Mock read() — return sinusoidal data for visualization
        def mock_read(number_of_samples_per_channel=-1):
            n_channels = len(task.channel_names)
            n_samples = 1024
            t = np.linspace(0, 1, n_samples)
            if n_channels == 1:
                # Single channel: return 1-D list (nidaqmx behavior)
                return list(np.sin(2 * np.pi * 5 * t) + 0.1 * np.random.randn(n_samples))
            else:
                # Multi-channel: return list of lists
                data = []
                for i in range(n_channels):
                    freq = 5 + i * 2
                    data.append(list(np.sin(2 * np.pi * freq * t) + 0.1 * np.random.randn(n_samples)))
                return data
        
        task.read = mock_read
        task.timing.cfg_samp_clk_timing = MagicMock()
        task.start = MagicMock()
        task.close = MagicMock()
        task.save = MagicMock()
        task.is_task_done = MagicMock(return_value=True)
        
        return task
    
    mock_nidaqmx.task.Task = create_mock_task
    mock_nidaqmx.Scale.create_lin_scale = MagicMock(return_value=MagicMock(name="scale_name"))
    
    # Install mock
    sys.modules['nidaqmx'] = mock_nidaqmx
    sys.modules['nidaqmx.constants'] = mock_constants
    sys.modules['nidaqmx.errors'] = MagicMock()
    sys.modules['nidaqmx.system'] = mock_nidaqmx.system
    sys.modules['nidaqmx.task'] = mock_nidaqmx.task
    
    print("Mock environment ready. All examples will run without hardware.")

## 1. Single Voltage Channel Acquisition

The simplest case: acquire voltage data from one analog input channel.

**Key points:**
- `AITask(task_name, sample_rate)` creates the task
- `add_channel()` configures the physical channel
- `start(start_task=True)` configures timing and starts acquisition
- `acquire_base()` returns shape `(n_channels, n_samples)` — transpose for plotting

In [ ]:
from nidaqwrapper import AITask
import matplotlib.pyplot as plt

# Create task with 25.6 kHz sample rate
task = AITask("voltage_acq", sample_rate=25600)

# Add voltage channel: Dev1/ai0, ±10V range
task.add_channel(
    channel_name="voltage_ch0",
    device_ind=0,         # First device in system
    channel_ind=0,        # Physical channel ai0
    units="V",            # Volts
    min_val=-10.0,
    max_val=10.0
)

# Configure timing and start acquisition
task.start(start_task=True)

# Read data (blocks until buffer has samples)
# Returns shape: (n_channels, n_samples)
data = task.acquire_base()

print(f"Acquired shape: {data.shape}")
print(f"Channels: {task.channel_list}")
print(f"Number of channels: {task.number_of_ch}")

# Plot: transpose to (n_samples, n_channels) for easier indexing
data_transposed = data.T
plt.figure(figsize=(10, 4))
plt.plot(data_transposed[:, 0])
plt.xlabel('Sample')
plt.ylabel('Voltage (V)')
plt.title('Single Channel Voltage Acquisition')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Clean up
task.clear_task()

## 2. Accelerometer (IEPE) Channel

Configure an IEPE accelerometer with sensitivity.

**IEPE sensors require:**
- `sensitivity` — sensor calibration value (e.g., 100.0 mV/g)
- `sensitivity_units` — "mV/g" or "mV/m/s**2"
- `units` — output units ("g" or "m/s**2")

The driver handles the conversion from voltage to engineering units.

In [ ]:
# Create task for vibration measurement
task = AITask("vibration_acq", sample_rate=25600)

# Add accelerometer channel with 100 mV/g sensitivity
task.add_channel(
    channel_name="accel_x",
    device_ind=0,
    channel_ind=0,
    sensitivity=100.0,           # Sensor sensitivity
    sensitivity_units="mV/g",    # Sensitivity units
    units="g",                   # Output in g's
    min_val=-5.0,
    max_val=5.0
)

task.start(start_task=True)
data = task.acquire_base()

print(f"Acquired {data.shape[1]} samples from {data.shape[0]} accelerometer channel")

# Plot acceleration time series
data_transposed = data.T
time = np.arange(data.shape[1]) / task.sample_rate

plt.figure(figsize=(10, 4))
plt.plot(time, data_transposed[:, 0])
plt.xlabel('Time (s)')
plt.ylabel('Acceleration (g)')
plt.title('Accelerometer Data Acquisition')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

task.clear_task()

## 3. Multi-Channel Acquisition

Acquire from multiple channels simultaneously. All channels are hardware-synchronized.

**Data shape convention:**
- `acquire_base()` returns `(n_channels, n_samples)` — nidaqmx native format
- Transpose to `(n_samples, n_channels)` for typical data analysis workflows

This example mixes voltage and accelerometer channels.

In [ ]:
# Create task for multi-channel acquisition
task = AITask("multi_channel", sample_rate=25600)

# Add three channels: voltage, accel X, accel Y
task.add_channel(
    channel_name="voltage",
    device_ind=0,
    channel_ind=0,
    units="V",
    min_val=-10.0,
    max_val=10.0
)

task.add_channel(
    channel_name="accel_x",
    device_ind=0,
    channel_ind=1,
    sensitivity=100.0,
    sensitivity_units="mV/g",
    units="g"
)

task.add_channel(
    channel_name="accel_y",
    device_ind=0,
    channel_ind=2,
    sensitivity=100.0,
    sensitivity_units="mV/g",
    units="g"
)

print(f"Configured channels: {task.channel_list}")
print(f"Total channels: {task.number_of_ch}")

task.start(start_task=True)
data = task.acquire_base()

print(f"\nAcquired data shape (channel-major): {data.shape}")

# Transpose to (n_samples, n_channels) for easier column access
data_transposed = data.T
print(f"Transposed shape (sample-major): {data_transposed.shape}")

# Plot all channels
time = np.arange(data.shape[1]) / task.sample_rate

fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

axes[0].plot(time, data_transposed[:, 0])
axes[0].set_ylabel('Voltage (V)')
axes[0].set_title('Channel 0: Voltage')
axes[0].grid(True, alpha=0.3)

axes[1].plot(time, data_transposed[:, 1])
axes[1].set_ylabel('Acceleration (g)')
axes[1].set_title('Channel 1: Accel X')
axes[1].grid(True, alpha=0.3)

axes[2].plot(time, data_transposed[:, 2])
axes[2].set_ylabel('Acceleration (g)')
axes[2].set_title('Channel 2: Accel Y')
axes[2].set_xlabel('Time (s)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

task.clear_task()

## 4. Context Manager (Recommended Pattern)

Use the `with` statement to ensure automatic cleanup even if an error occurs.

**Why use context managers:**
- Guaranteed resource cleanup
- Handles exceptions gracefully
- Cleaner, more Pythonic code

In [ ]:
# Context manager automatically calls clear_task() on exit
with AITask("context_example", sample_rate=25600) as task:
    # Configure channels
    task.add_channel(
        channel_name="ch0",
        device_ind=0,
        channel_ind=0,
        units="V"
    )
    
    task.add_channel(
        channel_name="ch1",
        device_ind=0,
        channel_ind=1,
        units="V"
    )
    
    # Start and acquire
    task.start(start_task=True)
    data = task.acquire_base()
    
    print(f"Acquired {data.shape[1]} samples from {data.shape[0]} channels")
    print(f"Mean values: {data.mean(axis=1)}")
    print(f"Std dev: {data.std(axis=1)}")

# Task is automatically cleaned up here, even if an exception occurred
print("\nTask cleaned up automatically via context manager.")

## 5. Custom Scaling

Apply a linear scale to voltage measurements (e.g., for custom sensors).

**Scaling options:**
- `scale=slope` — y = slope * x
- `scale=(slope, y_intercept)` — y = slope * x + y_intercept

When a scale is provided, `sensitivity` and `sensitivity_units` are not required.

In [ ]:
with AITask("scaled_input", sample_rate=25600) as task:
    # Custom sensor: voltage → pressure conversion
    # Scale: 1 V = 10 PSI, offset = 0
    task.add_channel(
        channel_name="pressure",
        device_ind=0,
        channel_ind=0,
        units="PSI",           # Custom engineering unit
        scale=10.0,            # Slope: 10 PSI per Volt
        min_val=0.0,
        max_val=100.0
    )
    
    # Another example: scale with offset
    # y = 5*x + 2
    task.add_channel(
        channel_name="temperature",
        device_ind=0,
        channel_ind=1,
        units="degC",
        scale=(5.0, 2.0),      # (slope, y_intercept)
        min_val=-40.0,
        max_val=125.0
    )
    
    task.start(start_task=True)
    data = task.acquire_base()
    
    data_transposed = data.T
    
    fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
    
    axes[0].plot(data_transposed[:, 0])
    axes[0].set_ylabel('Pressure (PSI)')
    axes[0].set_title('Scaled Channel: Pressure Sensor')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(data_transposed[:, 1])
    axes[1].set_ylabel('Temperature (°C)')
    axes[1].set_xlabel('Sample')
    axes[1].set_title('Scaled Channel: Temperature Sensor')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Pressure mean: {data_transposed[:, 0].mean():.2f} PSI")
    print(f"Temperature mean: {data_transposed[:, 1].mean():.2f} °C")

## 6. Manual Task Control

For advanced users: separate timing configuration from task start.

**Use case:** Configure timing, then start the task later via hardware trigger or explicit `task.start()`.

In [ ]:
task = AITask("manual_control", sample_rate=25600)

task.add_channel(
    channel_name="manual_ch",
    device_ind=0,
    channel_ind=0,
    units="V"
)

# Configure timing but don't start yet (start_task=False is default)
task.start(start_task=False)
print("Timing configured, task not started yet.")

# At this point, you could:
# - Configure hardware triggers on task.task
# - Wait for external event
# - Coordinate with other tasks

# Now start acquisition explicitly
task.task.start()
print("Task started explicitly.")

data = task.acquire_base()
print(f"Acquired {data.shape[1]} samples.")

task.clear_task()

## 7. Configuration Persistence (TOML)

Save task configuration to a human-readable TOML file for reuse.

**Benefits:**
- Hardware-portable: device names are aliased
- Version-controllable: plain text format
- Human-editable: change parameters without code

In [ ]:
import tempfile
import os

# Create a task configuration
with AITask("config_example", sample_rate=51200) as task:
    task.add_channel(
        channel_name="accel_z",
        device_ind=0,
        channel_ind=0,
        sensitivity=100.0,
        sensitivity_units="mV/g",
        units="m/s**2",
        min_val=-50.0,
        max_val=50.0
    )
    
    task.add_channel(
        channel_name="voltage_ref",
        device_ind=0,
        channel_ind=1,
        units="V",
        min_val=-5.0,
        max_val=5.0
    )
    
    # Save configuration to TOML
    config_path = os.path.join(tempfile.gettempdir(), "ai_config.toml")
    task.save_config(config_path)
    print(f"Configuration saved to: {config_path}\n")
    
    # Display the TOML content
    with open(config_path, 'r') as f:
        print("TOML Configuration:")
        print("-" * 50)
        print(f.read())
        print("-" * 50)

# Load configuration from TOML
print("\nLoading configuration from TOML...")
loaded_task = AITask.from_config(config_path)
print(f"Loaded task: {loaded_task.task_name}")
print(f"Sample rate: {loaded_task.sample_rate} Hz")
print(f"Channels: {loaded_task.channel_list}")

loaded_task.clear_task()

## 8. Wrapping External nidaqmx Task

Advanced: wrap a pre-configured `nidaqmx.Task` object.

**Use case:** Access low-level nidaqmx features not exposed by the wrapper API.

**Restrictions:**
- Channels must be configured before calling `from_task()`
- Timing must be configured before calling `from_task()`
- The wrapper will NOT close the task (caller retains ownership)

In [ ]:
import nidaqmx
from nidaqmx.constants import AcquisitionType, TerminalConfiguration, VoltageUnits

# Create and configure nidaqmx task directly
raw_task = nidaqmx.task.Task("external_task")
raw_task.ai_channels.add_ai_voltage_chan(
    physical_channel="Dev1/ai0",
    name_to_assign_to_channel="external_ch",
    terminal_config=TerminalConfiguration.DEFAULT,
    min_val=-10.0,
    max_val=10.0,
    units=VoltageUnits.VOLTS
)
raw_task.timing.cfg_samp_clk_timing(
    rate=25600,
    sample_mode=AcquisitionType.CONTINUOUS
)
raw_task.start()

# Wrap with AITask for high-level API
wrapped_task = AITask.from_task(raw_task)
print(f"Wrapped external task: {wrapped_task.task_name}")
print(f"Channels: {wrapped_task.channel_list}")
print(f"Sample rate: {wrapped_task.sample_rate} Hz")

# Use wrapper's acquire method
data = wrapped_task.acquire_base()
print(f"Acquired {data.shape[1]} samples from external task.")

# Important: caller must close the task
wrapped_task.clear_task()  # Issues warning, doesn't close
raw_task.close()           # Caller's responsibility
print("\nExternal task closed by caller.")

## Summary

**Key takeaways:**

1. **Task creation:** `AITask(task_name, sample_rate)`
2. **Channel types:** Voltage, accelerometer (IEPE), force (IEPE), custom-scaled
3. **Data shape:** `acquire_base()` returns `(n_channels, n_samples)` — transpose for typical workflows
4. **Context managers:** Use `with` for automatic cleanup
5. **Configuration:** Save/load TOML for hardware portability
6. **Advanced:** Wrap external nidaqmx tasks for low-level control

**Next steps:**
- See `02_analog_output.ipynb` for output generation with `AOTask`
- See `03_digital_io.ipynb` for digital I/O with `DITask`/`DOTask`
- See `04_advanced_multi_task.ipynb` for multi-task synchronization with `MultiHandler`